# Handling sequences with PyTorch

In [ ]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset

import torch.nn as nn
import torch.optim as optim
import torchmetrics

## Generating sequences

In [ ]:
def create_sequences(df, seq_length):
  xs, ys = [], []
  for i in range(len(df) - seq_length):
    x = df.iloc[i:(i+seq_length), 1]
    y = df.iloc[i+seq_length, 1]
    xs.append(x)
    ys.append(y)
  return np.array(xs), np.array(ys)

## Sequential Dataset

In [ ]:
train_data = pd.read_csv("data/train_data.csv")

X_train, y_train = create_sequences(train_data, 24*4)
print(X_train.shape, y_train.shape)

dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
)
print(len(dataset_train))

# Recurrent neural networks

## Building a forecasting RNN

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.RNN(
      input_size=1,
      hidden_size=32,
      num_layers=2,
      batch_first=True,
    )
    self.fc = nn.Linear(32, 1)

  def forward(self, x):
    h0 = torch.zeros(2, x.size(0), 32)
    out, _ = self.rnn(x, h0)  
    out = self.fc(out[:, -1, :])
    return out

# LSTM and GRU cells

## LSTM network

In [ ]:
class Net(nn.Module):
  def __init__(self, input_size):
    super().__init__()
    self.lstm = nn.LSTM(
      input_size=1,
      hidden_size=32,
      num_layers=2,
      batch_first=True,
    )
    self.fc = nn.Linear(32, 1)

  def forward(self, x):
    h0 = torch.zeros(2, x.size(0), 32)
    c0 = torch.zeros(2, x.size(0), 32)
    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])
    return out

## GRU network

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.gru = nn.GRU(
      input_size=1,
      hidden_size=32,
      num_layers=2,
      batch_first=True,
    )
    self.fc = nn.Linear(32, 1)

  def forward(self, x):
    h0 = torch.zeros(2, x.size(0), 32)
    out, _ = self.gru(x, h0)  
    out = self.fc(out[:, -1, :])
    return out

# Training and evaluating RNNs

## RNN training loop

In [ ]:
net = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(3):
  for seqs, labels in dataloader_train:
    seqs = seqs.view(16, 96, 1)
    outputs = net(seqs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

## Evaluating forecasting models

In [ ]:
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
  for seqs, labels in dataloader_test:
    seqs = seqs.view(32, 96, 1)
    outputs = net(seqs).squeeze()
    mse(outputs, labels)

test_mse = mse.compute()
print(f"Test MSE: {test_mse}")